# Tier 1: probing each target prefixes

In [4]:
import logging
import pickle
import uuid

from ping import PING
from measurements.src.atlas_api import get_measurements_from_tag
from utils.measurement_utils import load_pickle, save_config_file, get_prefix_from_ip
from default import (
    PROBES_FILE,
    ANCHORS_FILE,
    HITLIST_FILE,
    RIPE_CREDENTIALS,
    PREFIX_PROBE_VP,
)

logging.basicConfig(level=logging.INFO)

NB_TARGET = 2
NB_VP = 5

In [5]:
## load datasets

targets_per_prefix = load_pickle(HITLIST_FILE)

probes = load_pickle(PROBES_FILE)

anchors = load_pickle(ANCHORS_FILE)

## select targets and vps dataset

In [6]:
# select targets from anchors and vps from probes
targets = list(anchors.keys())[:NB_TARGET]
vps = {}
for i, probe in enumerate(probes):
    if i >= NB_VP:
        break
    vps[probe] = probes[probe]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 2
INFO:root:nb_vps : 5


## measure target prefixes

In [10]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type": "prefix",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
save_config_file(measurement_config)

pinger = PING(RIPE_CREDENTIALS)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_prefix(
    target_prefixes=target_prefixes,
    vps=vps,
    targets_per_prefix=targets_per_prefix,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
save_config_file(measurement_config)

INFO:root:measurement : 6cbaa66f-cb96-462c-8404-f29d37b6451b done


## retrieve prefix probing results 

In [16]:
measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))

# test that everything is alright
logging.info(f"nb measurements retrieved: {len(measurement_results)}")

print(f"measurements for {len(measurement_results)} target addr.")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 30:
        break
    print(target_addr, ":", len(results))

# save results
out_file = ANCHOR_PREFIX_PROBE_VP
print(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results, f)

INFO:root:nb measurements retrieved: 18


[{'fw': 4790, 'lts': 80, 'dst_name': '9f', 'af': 4, 'proto': 'ICMP', 'size': 48, 'result': [{'error': 'address not allowed'}], 'dup': 0, 'rcvd': 0, 'sent': 0, 'min': -1, 'max': -1, 'avg': -1, 'msm_id': 57505261, 'prb_id': 14, 'timestamp': 1689775119, 'msm_name': 'Ping', 'from': '95.232.13.43', 'type': 'ping', 'group_id': 57505261, 'step': None, 'stored_timestamp': 1689775120}, {'fw': 5080, 'mver': '2.6.2', 'lts': 67, 'dst_name': '9f', 'ttr': 42.814905, 'af': 4, 'proto': 'ICMP', 'size': 48, 'result': [{'error': 'dns resolution failed: nodename nor servname provided, or not known'}], 'dup': 0, 'rcvd': 0, 'sent': 0, 'min': -1, 'max': -1, 'avg': -1, 'msm_id': 57505261, 'prb_id': 60052, 'timestamp': 1689775120, 'msm_name': 'Ping', 'from': '45.138.229.91', 'type': 'ping', 'group_id': 57505261, 'step': None, 'stored_timestamp': 1689775120}, {'fw': 4790, 'lts': 84, 'dst_name': '9f', 'af': 4, 'proto': 'ICMP', 'size': 48, 'result': [{'error': 'dns resolution failed: nodename nor servname provide

AttributeError: 'list' object has no attribute 'items'

# Tier 2: probing each target

In [5]:
import logging
import pickle
import json
import uuid
import logging

from random import choice

from ping import PING
from measurements.src.atlas_api import get_measurements_from_id, get_measurements_from_tag
from default import (
    PROBES_FILE,
    ANCHORS_FILE,
    RIPE_CREDENTIALS,
    MEASUREMENT_CONFIG_PATH,
    TARGET_PROBE_VP,
)

logging.basicConfig(level=logging.INFO)


NB_TARGET = 2
NB_VP = 5

## select targets and vps dataset

In [6]:
# load probes
with open(PROBES_FILE, "rb") as f:
    probes = pickle.load(f)

# load anchors
with open(ANCHORS_FILE, "rb") as f:
    anchors = pickle.load(f)

In [7]:
# select targets from anchors
targets = set()
while len(targets) < NB_TARGET:
    random_target = choice(list(anchors))
    targets.add(random_target)
target = list(targets)

# select vps from probes
vps = {}
while len(vps) < NB_VP:
    random_vp = choice(list(probes))
    vps[random_vp] = probes[random_vp]

logging.info(f"nb targets: {len(targets)}")
logging.info(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 2
INFO:root:nb_vps : 5


## measure targets

In [ ]:
dry_run = True
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes as vps",
    "type": "target",
    "af": 4,
    "targets": target,
    "vps": vps,
}

# save measurement configuration before starting measurement
measurement_config_path = MEASUREMENT_CONFIG_PATH / f"{str(measurement_uuid)}.json"
with open(measurement_config_path, "w") as f:
    json.dump(measurement_config, f, indent=4)

pinger = PING(RIPE_CREDENTIALS)

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_target(
    targets=targets,
    vps=vps,
    tag=measurement_uuid,
    dry_run=dry_run
)

## retrieve traget probing results

In [ ]:
measurement_results = get_measurements_from_tag(measurement_config["UUID"])

if not measurement_results:
    for measurement_id in measurement_config["ids"]:
        logging.info(f"getting measurement with id : {measurement_id}")
        measurement_result = get_measurements_from_id(measurement_id)

logging.info(f"measurements for {len(measurement_results)} target addr")
for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    logging.info(f" {target_addr} : number of measurements {len(results)}")

# save results
out_file = ANCHOR_TARGET_PROBE_VP
logging.info(out_file)
with open(out_file, "wb") as f:
    pickle.dump(measurement_results, f)

## Tier 3: get passive measurements

In [ ]:
import pickle
import requests
import time

from collections import defaultdict
from json import JSONDecodeError

from default import ANCHORS_FILE

def get_measurement(url):
    response = requests.get(url).json()
    while True:
        for anchor in response["results"]:
            yield anchor

        if response["next"]:
            response = requests.get(response["next"]).json()
        else:
            break

anchors_file = ANCHORS_FILE
anchors_measurement_file = "../datasets/anchor_measurements_file.pickle"

with open(anchors_file, "rb") as f:
    anchors = pickle.load(f)

# load already existing measurements
try:
    with open(anchors_measurement_file, "rb") as f:
        anchor_measurements = pickle.load(f)
except FileNotFoundError:
    anchor_measurements = defaultdict(dict)


def get_measurement_result(url, max_retry: int = 60):
    for _ in range(max_retry):
        response = requests.get(url).json()
        if response:
            return response
        time.sleep(2)


print(f"{len(anchor_measurements)} probes were already treated")
try:
    for i, (anchor_addr, probe_description) in enumerate(anchors.items()):
        if i > 10:
            break

        if anchor_addr in anchor_measurements:
            continue

        anchor_measurements[anchor_addr] = defaultdict(list)

        print(f"getting measurements for {anchor_addr}")

        url = f"https://atlas.ripe.net/api/v2/measurements/ping/?target_ip={anchor_addr}"
        try:
            resp = requests.get(url, timeout=350).json()
        except JSONDecodeError:
            print(f"could not retreive results for probe: {anchor_addr}")
            time.sleep(5)
            continue

        for i, measurement in enumerate(resp['results']):

            # limit to ten meaasurements
            if i > 10:
                break

            url = measurement['result']
            results = get_measurement_result(url)

            for result in results:

                if 'src_addr' not in result:
                    continue

                # only keep measurement where there is more than one packet sent
                keys = [list(r.keys())[0] for r in result['result']]

                # extract RTTs from results
                rtt_list = []
                for r in result['result']:
                    try:
                        rtt_list.append(r['rtt'])
                    except KeyError:
                        continue
                print(rtt_list)

                # save RTT between the two probes
                try:
                    anchor_measurements[anchor_addr][result['src_addr']].extend(
                        rtt_list)
                except KeyError:
                    anchor_measurements[anchor_addr][result['src_addr']] = rtt_list

            time.sleep(2)

    print(f"atlas measurement retreived for {len(anchor_measurements)} probes")

except KeyboardInterrupt:
    print("interrupted")
    pass

with open(anchors_measurement_file, "wb") as f:
    pickle.dump(anchor_measurements, f)


import pickle
import requests
import time

from collections import defaultdict

anchors_file = "../datasets/anchors.pickle"
anchors_measurement_file = "../datasets/anchor_measurements_file.pickle"

with open(anchors_file, "rb") as f:
    anchors = pickle.load(f)

# load already existing measurements
try:
    with open(anchors_measurement_file, "rb") as f:
        anchor_measurements = pickle.load(f)
except FileNotFoundError:
    anchor_measurements = defaultdict(dict)

starting_point = 52592569
starting_point = 52592469
starting_point = 52592369
starting_point = 52592269
starting_point = 52592069
starting_point = 52591869
starting_point = 52591669

starting_point = 52591613
end_point = 52592569


def get_results(measurement_id, max_retry: int = 10):
    for _ in range(max_retry):
        response = requests.get(
            "https://atlas.ripe.net/api/v2/"
            f"measurements/{measurement_id}/results/"
        ).json()

        if response:
            return response
        time.sleep(2)


print(f"{len(anchor_measurements)} probes were already treated")
try:
    for id in reversed(range(starting_point, end_point)):
        print(id)

        id_done = []
        for dst_addr in anchor_measurements:
            id_done.extend(anchor_measurements[dst_addr]["id"])

        if id in id_done:
            print("id:", id, "already retrieved")
            continue

        response = get_results(id)
        measurement_results = []

        if not response:
            continue

        # parse response
        for result in response:

            if type(result) is str:
                print(result)
                continue
            # parse results and calculate geoloc
            if result.get('result') is not None:

                dst_addr = result['dst_addr']
                vp_ip = result['from']

                if type(result['result']) == list:
                    rtt_list = [list(rtt.values())[0]
                                for rtt in result['result']]
                else:
                    rtt_list = [result['result']["rtt"]]

                # remove stars from results
                rtt_list = list(filter(lambda x: x != "*", rtt_list))
                if not rtt_list:
                    continue

                # get min rtt
                min_rtt = min(rtt_list)
                if type(min_rtt) is str:
                    print(min_rtt)
                    continue

                # both vp and target coordinates
                try:
                    vp_lat = anchors[vp_ip]['latitude']
                    vp_lon = anchors[vp_ip]['longitude']
                except KeyError:
                    continue

                measurement_results.append({
                    "node": vp_ip,
                    "min_rtt": min_rtt,
                    "rtt_list": rtt_list,
                    "vp_lat": vp_lat,
                    "vp_lon": vp_lon,
                })

            else:
                print(f"no results: {result}")

        measurement_results = sorted(
            measurement_results, key=lambda x: x["min_rtt"])
        print(dst_addr, ":", len(measurement_results), measurement_results)
        anchor_measurements[dst_addr].append(measurement_results)

        print(len(anchor_measurements[dst_addr]["id"]))

except KeyboardInterrupt:
    pass
with open(anchors_measurement_file, "wb") as f:
    pickle.dump(anchor_measurements, f)